In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part'):
    shutil.rmtree('./file/2.part')

os.makedirs('./file/2.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.url/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = input_.loc[a, 'Url']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            market = html.xpath('//div[@class="part__canada-ligne"]/img/@title')[0].strip() if html.xpath('//div[@class="part__canada-ligne"]/img') else ''
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =
                
                type_ = ''.join([text.strip() for text in html.xpath('//div[@class="part__canada-ligne"]/text()')]).strip()

                # = = = = = = = = = = = = = = =

                dict_src, list_src = {}, [f'https://ecat.spectrapremium.com{src.strip()}' for src in html.xpath('//a[@class="part__magic-scroll-images__thumbnail"]/@href')]
                while 'https://ecat.spectrapremium.com#' in list_src:
                    list_src.remove('https://ecat.spectrapremium.com#')
                
                for i, src in enumerate(list_src):
                    dict_src[str(i)] = src

                # = = = = = = = = = = = = = = =

                part_number = html.xpath('//h1[@class="part__titre-code"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                list_div, dict_count = html.xpath('//dl[@class="part__specifications"]/div[@class="part__nom-spec-valeur-spec"]'), {}
                for div in list_div:
                    key = div.xpath('./dt[@class="part__nom-spec"]/text()')[0].strip()
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                        dict_count[key] = 1
                
                dict_specification, dict_no = {}, {}
                for i, div in enumerate(list_div):
                    key = div.xpath('./dt[@class="part__nom-spec"]/text()')[0].strip()
                    if dict_count[key] == 1:
                        dict_specification[str(i)] = {key: div.xpath('./dd[@class="part__valeur-spec"]/text()')[0].strip()}
                    else:
                        if key in dict_no:
                            dict_no[key] += 1
                        else:
                            dict_no[key] = 1
                        dict_specification[str(i)] = {f'''{key}-{dict_no[key]}''': div.xpath('./dd[@class="part__valeur-spec"]/text()')[0].strip()}
                
                # = = = = = = = = = = = = = = =

                vehicle_page = int(html.xpath('//li[@class="paginator__last"]/a/@href')[0].strip().removesuffix('#applications').split('=')[-1]) if html.xpath('//li[@class="paginator__last"]') else 1

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Market': market,
                                         'Type': type_,
                                         'Part Number': part_number,
                                         'Vehicle': '',
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Specification': json.dumps(dict_specification),
                                         'Vehicle Page': vehicle_page}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part/{file}',
              f'''./file/2.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://ecat.spectrapremium.com/en/parts/BMP08A
[剩余数量：5] - [当前时间：10:25:34]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.https://ecat.spectrapremium.com/en/parts/BMP06A
[剩余数量：4] - [当前时间：10:25:34]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://ecat.spectrapremium.com/en/parts/BMP03A
[剩余数量：3] - [当前时间：10:25:34]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://ecat.spectrapremium.com/en/parts/BMP05A
[剩余数量：2] - [当前时间：10:25:34]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 8.https://ecat.spectrapremium.com/en/parts/BMP11A
[剩余数量：1] - [当前时间：10:25:35]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 9.https://ecat.spectrapremium.com/en/parts/BMP13A
[剩余数量：0] - [当前时间：10:25:36]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 7.https://ecat.spectrapremium.com/en/parts/BMP10A
[剩余数量：0] - [当前时间：10:25:36]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 10.https://ecat.spectrapremium.com/en/parts/BMP14A
[剩余数量：0] - [当前时间：10:25:37]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://ecat.spectrapre

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3945.72it/s]

Done ~
